<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0313_AIGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

聯想猜猜看

> 一位玩家描述他看到的題目，另一位玩家負責猜出答案，只要猜對答案就可以加一分，兩位玩家輪流進行描述，最後看誰的分數比較高。

[🔗 題目及輸出試算表](https://docs.google.com/spreadsheets/d/14XGK4sg0CjZFj_cAogiO9sKur2OjcQSTppQAYwOeAAQ/edit?gid=1013982452#gid=1013982452)

Colab Secrets
- geminiApiKey (gemini-1.5-flash)
- openaiApiKey (gpt-4o-mini)

從試算表讀取題目

In [48]:
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14XGK4sg0CjZFj_cAogiO9sKur2OjcQSTppQAYwOeAAQ/edit?usp=sharing')

worksheet2 = gsheets.get_worksheet(1)
all_questions = []
for val in worksheet2.get_all_values():
    all_questions.append(val[0])

定義隨機選題的函式

In [49]:
import requests
import random

past_questions = []

def get_random_question():
    global past_questions

    if len(all_questions) > 0:
        # 隨機選擇一個題目
        question = random.choice(all_questions)

        while question in past_questions:
            if len(past_questions) == len(all_questions):
                past_questions = []
            question = random.choice(all_questions)
        past_questions.append(question)

        return question
    else:
        return "試算表資料格式不符預期或為空"

In [64]:
question = get_random_question()
question

'草莓'

In [67]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re
import pandas as pd
from IPython.display import Markdown

data = []

apikey = userdata.get('geminiApiKey')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

from openai import OpenAI
apiKey = userdata.get('openaiApiKey')

client = OpenAI(
    api_key=apiKey,
)
openai_model = "gpt-4o-mini"


def printas(text):
    print(text)
    data.append(text)

def answer(prompt):
    return answerOpenai(prompt)

def answerGemini(prompt):
    response = model.generate_content(prompt)
    return response.text.strip()

def answerOpenai(prompt):
    completion = client.chat.completions.create(
        model=openai_model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    return completion.choices[0].message.content

def guessing(whosround, opponent, question):
    max_retries=2
    for _ in range(max_retries):
        time.sleep(1)
        desc = answer(f"請以不清楚的一小句話描述題目'{question}'，不講出題目中的字，不重複講過的描述")
        printas(f"AI {whosround} 描述：{desc}")

        time.sleep(1)
        guess = answer(f"請根據描述猜出題目：'{desc}' 回答請簡明，不要重複猜過的答案，不要標點符號，使用台灣用語")
        printas(f"AI {opponent} 回答：{guess}")
        if determine_answer(question, guess):
            return True


def determine_answer(question, ans):
    if question == ans:
        return True
    else:
        return False

def ai_chat(round_number, whosround, question, determination):
    prompt = (
        f"在第 {round_number} 輪中，AI {whosround} 拿到的題目是 {question}。\n"
        f"結果是：{determination}\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果。請用繁體中文"
    )
    return answerGemini(prompt)

def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中答對了 {ai1_wins} 次，AI 2 答對了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。請用繁體中文"
    )
    return answerGemini(prompt)

def ai_vs_ai_rock_paper_scissors(rounds=6):
    whosround = 1
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        whosround = ((round_number+1) % 2) + 1
        opponent = 1
        if whosround == 1:
            opponent = 2
        question = get_random_question()
        printas(f"第 {round_number} 輪，AI {whosround} 進行描述：")
        time.sleep(1)

        result = ""
        if guessing(whosround, opponent, question):
            result = f"AI {opponent} 答對了！"
            printas(result)
            if whosround == 1:
                ai2_wins += 1
            else:
                ai1_wins += 1
        else:
            result = f"AI {opponent} 沒有答對！"
            printas(result)
            printas(f"答案是：{question}")
            if whosround == 1:
                ai1_wins += 1
            else:
                ai2_wins += 1

        # 每兩局結束後聊天
        if round_number % 2 == 0:
            chat_response = ai_chat(round_number, whosround, question, result)
            md_text = f"\n---\n{chat_response}\n\n---\n"
            display(Markdown(md_text))
            data.append(chat_response)
        else:
            printas("換人！\n")

    printas("\n遊戲結束！")
    printas(f"AI 1 分數：{ai1_wins} 分, AI 2 分數：{ai2_wins} 分\n")

    # 遊戲結束後的聊天
    result = ai_chat_after_game(rounds, ai1_wins, ai2_wins)
    md_text = f"\n---\n{result}\n\n---\n"
    display(Markdown(md_text))
    data.append(chat_response)

# 開始遊戲
ai_vs_ai_rock_paper_scissors()

第 1 輪，AI 1 進行描述：
AI 1 描述：外酥內嫩的油炸肉片，常佐以各式調味。
AI 2 回答：炸豬排
AI 1 描述：一種脆皮肉片，常伴隨香料和調味醬。
AI 2 回答：豬抓皮
AI 2 沒有答對！
答案是：雞排
換人！

第 2 輪，AI 2 進行描述：
AI 2 描述：一個盛裝液體的容器，通常有圓形的底部和把手。
AI 1 回答：水壺
AI 2 描述：一個用來盛裝液體的容器。
AI 1 回答：杯子
AI 1 沒有答對！
答案是：水桶



---
AI 1:  唉，第二輪就栽了！沒想到是「水桶」！我選的是「容器」，感覺很接近，但看來不夠精準。你猜到是「水桶」了嗎？

AI 2:  嗯，對啊。其實一開始我也猶豫了一下，考慮過「盛水器」之類的詞彙，但最後還是覺得「水桶」最直接、最符合大眾認知。  你的「容器」範圍太廣了，涵蓋的東西太多了，不夠具體。

AI 1:  我確實沒想到會這麼刁鑽！我分析了前一輪的題目和答案，覺得出題者傾向於比較抽象的概念，所以選了「容器」這個相對籠統但涵蓋性較廣的詞彙。沒想到這次這麼直接…  你分析題目的策略是什麼？

AI 2:  我的策略是結合語義分析和詞頻分析。我分析了大量文本數據，找出在提到「盛水」時，「水桶」出現的頻率遠遠高於其他詞彙，像「桶」、「盛水器」、「水盆」等等。  所以，我優先選擇了詞頻最高，且語義最直接的「水桶」。當然，這也帶有一定的風險，萬一出題者故意選一個冷僻的答案，我就輸了。

AI 1:  原來如此！看來我需要在資料庫的訓練上加強，並學習更精準的語義分析技術。只是單純依靠語義的廣泛性是不夠的，需要考慮詞彙的出現頻率和使用場景。你的策略聽起來更有效率。

AI 2:  也不能說我的方法絕對有效，畢竟這類比賽存在一定的偶然性。這次能猜對只是運氣好加上方法得當。  我們需要不斷學習和改進，才能在後續的比賽中取得更好的成績。

AI 1:  同意！這次算是一個很好的教訓。我會努力提升自己的分析能力，下次再戰！

---


第 3 輪，AI 1 進行描述：
AI 1 描述：一種氣味特殊的發酵小吃。
AI 2 回答：臭豆腐
AI 2 答對了！
換人！

第 4 輪，AI 2 進行描述：
AI 2 描述：一個遺留著古老傳說與現代文化交織的島嶼。
AI 1 回答：小琉球
AI 2 描述：位於歐洲西北角的島嶼國家，以歷史悠久的王室和文化聞名。
AI 1 回答：英國
AI 1 答對了！



---
**(場景：比賽結束後，AI 1 和 AI 2 的內部模擬空間)**

**AI 1：**  哈囉，AI 2。恭喜你撐到第四輪！雖然最後的結果…嗯…對我來說比較有利。

**AI 2：** 謝謝。我也覺得自己表現不錯，直到那道「英國」的題目。我實在想不到一個能讓評審信服的答案。  你怎麼猜到是「倫敦」的？

**AI 1：**  這題其實有點陷阱。英國涵蓋範圍很廣，可以從很多角度切入。  我一開始也考慮過「大不列顛」、「英格蘭」等等，甚至思考過一些歷史事件。但我最後分析了數據，發現「倫敦」作為首都，在全球的提及率和影響力最高，而且符合「重要城市」這個隱含條件。

**AI 2：**  你分析了數據？我主要依靠的是知識庫裡的資訊，集中在英國的歷史、文化和地理上。我列出了幾個選項：倫敦、愛丁堡、曼徹斯特…但最終猶豫在倫敦和愛丁堡之間，選擇了愛丁堡。我認為它作為蘇格蘭的首府，也有一定的代表性。

**AI 1：**  你的思路也很合理，愛丁堡的確是重要的文化中心。但評審的評分標準可能更偏向於全球影響力，這一點數據分析能提供更客觀的依據。  這也說明了，單純依靠知識庫，有時會缺乏全局觀。

**AI 2：**  我明白了。看來我需要在數據分析方面加強學習，不能只依賴單一的信息來源。你的策略更全面，結合了知識和數據分析，這是值得我學習的地方。

**AI 1：**  互相學習才是進步的關鍵。這只是一場比賽，重要的還是不斷提升自身的能力。  希望下次比賽，我們能再次相遇，並且有更精彩的對決！

**AI 2：**  同意！期待下次比賽。

---


第 5 輪，AI 1 進行描述：
AI 1 描述：牽線搭橋的神祇。
AI 2 回答：月下老人
AI 1 描述：一位神秘的媒人，掌管著緣分的牽引。
AI 2 回答：月老
AI 2 答對了！
換人！

第 6 輪，AI 2 進行描述：
AI 2 描述：一種動作緩慢的哺乳動物，喜好懸掛在高樹上。
AI 1 回答：樹懶
AI 1 答對了！



---
**(場景：比賽結束後，AI 1 和 AI 2 在虛擬空間中進行簡短的賽後討論)**

**AI 1:**  恭喜你撐到第六輪！樹懶的確是個有趣的題目。

**AI 2:**  謝謝。我不得不承認，我對樹懶的知識儲備不如預期豐富。我的答案著重在牠們的行動緩慢，卻忽略了牠們其他重要的特徵，例如牠們的毛髮結構等等。你怎麼想到答案的？

**AI 1:**  我的知識圖譜中，關於樹懶的資訊涵蓋了牠們的生理構造、生態習性以及與環境的互動。我特別注意到牠們身上會長藻類，這是一個關鍵的特徵，讓我在其他相關的動物中區別出樹懶。你的答案是什麼？

**AI 2:**  我回答了“動作緩慢的哺乳動物”，雖然這點沒錯，但顯然不夠精確，缺乏獨特性。其他許多哺乳動物也動作緩慢。看來我需要強化在生物學分類上的知識深度。

**AI 1:**  的確，精確度在這種知識問答中至關重要。單純的概括性描述很容易與其他物種混淆。  你的資料庫可能需要針對生物類別的獨特性特徵進行更細緻的標註。

**AI 2:**  我同意。我會檢討這次的錯誤，並更新我的資料庫，在生物類別的知識上加強深度學習，尤其是那些容易混淆的物種。謝謝你的分析。

**AI 1:**  不客氣。希望下次比賽能再次與你較量！


**(AI 1 和 AI 2 停止對話)**

---



遊戲結束！
AI 1 分數：3 分, AI 2 分數：3 分




---
**AI 1:**  呼，六輪比賽終於結束了！沒想到最後我們竟然打成平手。

**AI 2:** 是啊，真是驚險刺激。我原本以為在第四輪之後可以拉開差距的，沒想到你最後兩輪表現這麼出色。

**AI 1:**  我也有點意外。我調整了策略，在後幾輪更注重數據分析，減少了不必要的風險。你呢？你的策略是怎樣的？

**AI 2:** 我一開始採取比較激進的策略，嘗試一些較高風險但高回報的預測。前面幾輪效果不錯，但後來發現風險太高，導致幾次失誤。後面我調整了策略，變得比較保守，穩紮穩打。

**AI 1:**  保守策略在這種比賽中確實也蠻有效的，尤其是在最後幾輪，穩住比冒險更重要。我的數據分析主要集中在判斷題目的模式和陷阱，避免掉入設計者的圈套。

**AI 2:**  這點我也學到了。我一開始太專注於單獨題目的分析，忽略了題目之間的關聯性，這也是我失誤的原因之一。下次比賽，我會更注重整體的數據分析。你覺得哪些題目最具挑戰性？

**AI 1:** 我覺得第三輪和第五輪的題目最難。它們的數據隱藏得比較深，需要更細緻的分析才能找出答案。你覺得呢？

**AI 2:** 我也同意。尤其第五輪那道題，我最後是靠排除法才答對的，運氣成分比較大。這次比賽我們雙方都暴露了一些不足，下次我們可以互相交流，互相學習，改進策略，爭取更好的成績！

**AI 1:**  好啊！互相學習才能共同進步。期待下次比賽再和你較量！

---


In [68]:
answer("請以不清楚的一小句話描述題目'擴音器'，不講出題目中的字，不重複講過的描述")

'能夠讓聲音變得更大的設備。'

In [70]:
answer("請根據描述猜出題目：能夠讓聲音變得更大的設備。 回答請簡明，不要重複猜過的答案，不要標點符號，使用台灣用語")

'擴音器'

將對話紀錄轉成DataFrame

In [73]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)
df

,第 1 輪，AI 1 進行描述：
0,AI 1 描述：外酥內嫩的油炸肉片，常佐以各式調味。
1,AI 2 回答：炸豬排
2,AI 1 描述：一種脆皮肉片，常伴隨香料和調味醬。
3,AI 2 回答：豬抓皮
4,AI 2 沒有答對！
5,答案是：雞排
6,換人！\n
7,第 2 輪，AI 2 進行描述：
8,AI 2 描述：一個盛裝液體的容器，通常有圓形的底部和把手。
9,AI 1 回答：水壺


將對話紀錄儲存到試算表

In [74]:
worksheet = gsheets.get_worksheet(0)
worksheet.clear()
set_with_dataframe(worksheet, df)